# Entity Extraction of Pentagon Papers

In this notebook we'll use [```spaCy```, a Python module that does natural language processing,](https://urldefense.com/v3/__https://spacy.io/usage/models__;!!DZ3fjg!qvPffHhHBJIodW8Tf1IP52IiCn_x6Tj9b547HUdN1URaf5JTSz-yueHuvNkYDRWVgsk$ ) including part-of-speech tagging and named entity recognition (NER).

In [1]:
!pip install spacy                          
# this gets the python module itself

In [2]:
!python -m spacy download en_core_web_sm    
# this gets a particular English-lang model
# if this doesn't work try saying !python3 (everything else the same)
# or run the cell below and check what version you're running, and then say
# !python3.7, or !python3.8 (with everything else the same) as appropriate.

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.1.0/en_core_web_sm-3.1.0-py3-none-any.whl (13.6 MB)
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
from platform import python_version
print(python_version())

3.8.8


In [4]:
import spacy
from spacy import displacy
from pprint import pprint
from collections import Counter
import en_core_web_sm
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import dataframe_image as dfi
%matplotlib inline

In [5]:
engl_nlp = en_core_web_sm.load()

### SECTION 1: Ingest and Chunk the File



<H2> Main Data Frame </H2>
    
Notes: The table does not have the text file "Pentagon-Papers-Part-IV-C-10.txt" because this entire file is all statistical reports and maps that do not translate to NLP. This leaves the entire text file corpus at 48 documents, each located in its entirity in the text column in the df. 
    
Overall_years refers to the span of dates that the content within is listed to pertain to. This information was found via the index (first row of df). Some documents did not have listed dates, so the information was found by searching through the body of the document, and often was found in the files chronology section.
    
Specific_dates is a more granular look at the time span a particular document refers to than overall_years. Most of the information is in month/year-month/year format, but for documents where it refered to a very specific start and end dates (such as Part-V-A which documents the U.S. governments official statements of position on the war to the public) then the date format is month/day/year-month/day/year. 

In [6]:
#use pwd to see where I am currently in the directory if I am having trouble identifying where I am
path = Path('full_txt_document.csv')
main_df = pd.read_csv(path, sep = ',')

In [7]:
main_df = main_df.drop(["Unnamed: 0"], axis=1)
main_df

,name,title,overall_years,specific_dates,text
0,Index,Index,1945-1967,1945-1967,Final Report OSD Vietnam Task Force (u) w1 In...
1,Part-I,Vietnam and the U.S.,1940-1950,1940-1950,EXECUTIVE SECRETARIAT TLE JJ RMM KDA ; r SS-...
2,Part-2,U.S. Involvement in the Franco-Viet Minh War,1950-1954,1950-1954,II U.S. Involvement in the Franco-Viet Minh W...
3,Part-3,The Geneva Accords,1954,1954,III The Geneva Accords 1954 A. U.S. Military ...
4,Part-IV-A-1,NATO and SEATO - A Comparison,1954-1960,1954-1960,", IwA Evolution of the War (26 Vols.) U.S. MAP..."
5,Part-IV-A-2,Aid for France in Indochina,1950-1954,1950-1954,A Evolution of the War (26 Vols.) U.S. MAP for...
6,Part-IV-A-3,U.S. and France's Withdrawl from Vietnam,1954-1956,1954-1956,A Evolution of the War (26 Vols.) U.S. MAP for...
7,Part-IV-A-4,U.S. Training of Vietnamese National Army,1954-1959,1954-1959,", Iw A Evolution of the War (26 Vols.) U.S. MA..."
8,Part-IV-A-5,Origins of the Insurgency,1954-1960,1954-1960,", IwA Evolution of the War (26 Vols.) U.S. MAP..."
9,Part-IV-B-1,The Kennedy Commitments and Programs,1961,1961,", IwB Evolution of the War (26 Vols.) Counteri..."


In [8]:
list_of_docs = main_df['name'].to_list()
list_of_docs

['Index',
 'Part-I',
 'Part-2',
 'Part-3',
 'Part-IV-A-1',
 'Part-IV-A-2',
 'Part-IV-A-3',
 'Part-IV-A-4',
 'Part-IV-A-5',
 'Part-IV-B-1',
 'Part-IV-B-2',
 'Part-IV-B-3',
 'Part-IV-B-4',
 'Part-IV-B-5',
 'Part-IV-C-1',
 'Part-IV-C-2a',
 'Part-IV-C-2b',
 'Part-IV-C-2c',
 'Part-IV-C-3',
 'Part-IV-C-4',
 'Part-IV-C-5',
 'Part-IV-C-6a',
 'Part-IV-C-6b',
 'Part-IV-C-6c',
 'Part-IV-C-7a',
 'Part-IV-C-7b',
 'Part-IV-C-8',
 'Part-IV-C-9a',
 'Part-IV-C-9b',
 'Part-V-A-Vol-IA',
 'Part-V-A-Vol-IB',
 'Part-V-A-Vol-IC',
 'Part-V-A-Vol-IID',
 'Part-V-B-1',
 'Part-V-B-2a',
 'Part-V-B-2b',
 'Part-V-B-3a',
 'Part-V-B-3b',
 'Part-V-B-3c',
 'Part-V-B-3d',
 'Part-V-B-4-Book-I',
 'Part-V-B-4-Book-II',
 'Part-VI-A',
 'Part-VI-B',
 'Part-VI-C-1',
 'Part-VI-C-2',
 'Part-VI-C-3',
 'Part-VI-C-4']

In [8]:
#create df without text column and export as a jpg image
metadata_df = main_df.drop(["text"], axis=1)
metadata_df
dfi.export(metadata_df, 'metadata.jpg')

<H2> Functions to Chunk the Document, Extract the Entities, and Create a DataFrame from the Entity Dictionary</H2>

In [10]:
#function to prepare the document for extraction
#takes input of the file path, chunks into 300 words, and combines those 300 words into one string

def prepare_document(file_path):
    combined_word_lists = []
    total_lists = []

    path = Path(file_path)
    input_doc = open(path, encoding = 'utf-8').read()

    words = input_doc.split()

    for i in range(0, len(words), 300):
        chunk_words = words[i: i+300]
        combined_word_lists.append(chunk_words)
    
    #make every 300 words into a single string that is an individual element within the list 'total_lists'
    
    for list in combined_word_lists: 
        list_into_string = " ".join([word for word in list])
        total_lists.append(list_into_string)

    return total_lists

In [11]:
#function that does entity extraction on the prepared document using spaCy
#input: formatted document from prepare_document
#output: entity extraction dictionary

def entity_extraction(individual_document):
    ent_types = dict()

    for i in individual_document: #for each string in the list
        doc = engl_nlp(i)

        for entity in doc.ents:           # for each character, go through all the entities
            label = entity.label_         # get their labels
            if label not in ent_types:    # make sure there's a key for this label in the dictionary
                ent_types[label] = Counter()      # each label key points to a Counter for examples
            text = entity.text
            ent_types[label][text] += 1 
    
    return ent_types

In [12]:
def org_entity(document_dictionary):

    # for each dictionary, create a new df
    new_df_org = pd.DataFrame(document_dictionary.items(), columns =["Organizations", "Count"])
    new_df_org.sort_values(by=["Organizations"], inplace = True)
    part_III_orgs_alpha = new_df_org
    new_df = part_III_orgs_alpha.reset_index(drop=True)
    
    return new_df

In [71]:
def person_entity(document_dictionary):

    # for each dictionary, create a new df
    new_df_person = pd.DataFrame(document_dictionary.items(), columns =["Person", "Count"])
    new_df_person.sort_values(by=["Person"], inplace = True)
    part_III_person_alpha = new_df_person
    new_df_person = part_III_person_alpha.reset_index(drop=True)
    
    return new_df_person

In [14]:
# concat dictionary to existing main dictionary

def concat_df(main_df, new_df):

    main_df= pd.concat([main_df, new_df], axis = 1)
    
    return main_df

<H2> Formatting DF's for Organizations </H2>

In [15]:
part_index_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-Index.txt')
part_index_dictionary_org = entity_extraction(part_index_formatted_org)
part_index_org = org_entity(part_index_dictionary_org['ORG'])
# print(part_index_org)

In [16]:
part_I_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-I.txt')
part_I_dictionary_org = entity_extraction(part_I_formatted_org)
part_I_org = org_entity(part_I_dictionary_org['ORG'])
# print(part_I_org)
# print(len(part_I_org))

In [17]:
part_II_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-II.txt')
part_II_dictionary_org = entity_extraction(part_II_formatted_org)
part_II_org = org_entity(part_II_dictionary_org['ORG'])
# print(part_II_org)
# print(len(part_II_org))

In [18]:
#concat dataframes together
main_df = concat_df(part_index_org, part_I_org)
main_df = concat_df(main_df, part_II_org)

In [19]:
part_III_formatted = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-III.txt')
part_III_dictionary = entity_extraction(part_III_formatted)
part_III_orgs = org_entity(part_III_dictionary['ORG'])
main_df = concat_df(main_df, part_III_orgs)

In [20]:
part_iv_a_1_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-1.txt')
part_iv_a_1_dictionary_org = entity_extraction(part_iv_a_1_formatted_org)
part_iv_a_1_org = org_entity(part_iv_a_1_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_a_1_org)

In [21]:
main_df

,Organizations,Count,Organizations,Count,Organizations,Count,Organizations,Count,Organizations,Count
0,A. Public Statements,1.0,A. Public Statements,1.0,8 1954,1.0,A-16 a.,1,83d Congress,2.0
1,A. U.S. France,1.0,A. U.S. France,1.0,A-14 DEPARTMENT OF,1.0,A-24 4,1,A-19 d. UN,1.0
2,A. U.S. Military Planning,1.0,A. U.S. Military Planning,1.0,A-17 d. French Intransigence,1.0,A-25 b. French,1,A-29(O,1.0
3,ASDISA,1.0,ASDISA,1.0,A-35 b. Military Assistance Program,1.0,A-25 c. United Action,1,A-3448,1.0
4,Announced Position Statements C. Histories of ...,1.0,Announced Position Statements C. Histories of ...,1.0,A-35 c.,1.0,A-34 a. United,1,A-7,1.0
...,...,...,...,...,...,...,...,...,...,...
358,NaN,NaN,NaN,NaN,NaN,NaN,the united a.ction option,1,NaN,NaN
359,NaN,NaN,NaN,NaN,NaN,NaN,the united action,2,NaN,NaN
360,NaN,NaN,NaN,NaN,NaN,NaN,the united action concept of,1,NaN,NaN
361,NaN,NaN,NaN,NaN,NaN,NaN,united action,2,NaN,NaN


In [22]:
part_iv_a_2_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-2.txt')
part_iv_a_2_dictionary_org = entity_extraction(part_iv_a_2_formatted_org)
part_iv_a_2_org = org_entity(part_iv_a_2_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_a_2_org)

In [23]:
part_iv_a_3_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-3.txt')
part_iv_a_3_dictionary_org = entity_extraction(part_iv_a_3_formatted_org)
part_iv_a_3_org = org_entity(part_iv_a_3_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_a_3_org)

In [24]:
part_iv_a_4_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-4.txt')
part_iv_a_4_dictionary_org = entity_extraction(part_iv_a_4_formatted_org)
part_iv_a_4_org = org_entity(part_iv_a_4_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_a_4_org)

In [25]:
part_iv_a_5_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-5.txt')
part_iv_a_5_dictionary_org = entity_extraction(part_iv_a_5_formatted_org)
part_iv_a_5_org = org_entity(part_iv_a_5_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_a_5_org)

In [26]:
part_iv_b_1_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-1.txt')
part_iv_b_1_dictionary_org = entity_extraction(part_iv_b_1_formatted_org)
part_iv_b_1_org = org_entity(part_iv_b_1_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_b_1_org)

In [27]:
part_iv_b_2_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-2.txt')
part_iv_b_2_dictionary_org = entity_extraction(part_iv_b_2_formatted_org)
part_iv_b_2_org = org_entity(part_iv_b_2_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_b_2_org)

In [28]:
part_iv_b_3_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-3.txt')
part_iv_b_3_dictionary_org = entity_extraction(part_iv_b_3_formatted_org)
part_iv_b_3_org = org_entity(part_iv_b_3_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_b_3_org)

In [29]:
part_iv_b_4_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-4.txt')
part_iv_b_4_dictionary_org = entity_extraction(part_iv_b_4_formatted_org)
part_iv_b_4_org = org_entity(part_iv_b_4_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_b_4_org)

In [30]:
part_iv_b_5_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-5.txt')
part_iv_b_5_dictionary_org = entity_extraction(part_iv_b_5_formatted_org)
part_iv_b_5_org = org_entity(part_iv_b_5_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_b_5_org)

In [31]:
part_iv_c_1_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-1.txt')
part_iv_c_1_dictionary_org = entity_extraction(part_iv_c_1_formatted_org)
part_iv_c_1_org = org_entity(part_iv_c_1_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_1_org)

In [32]:
part_iv_c_2a_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-2a.txt')
part_iv_c_2a_dictionary_org = entity_extraction(part_iv_c_2a_formatted_org)
part_iv_c_2a_org = org_entity(part_iv_c_2a_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_2a_org)

In [33]:
part_iv_c_2b_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-2b.txt')
part_iv_c_2b_dictionary_org = entity_extraction(part_iv_c_2b_formatted_org)
part_iv_c_2b_org = org_entity(part_iv_c_2b_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_2b_org)

In [34]:
part_iv_c_2c_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-2c.txt')
part_iv_c_2c_dictionary_org = entity_extraction(part_iv_c_2c_formatted_org)
part_iv_c_2c_org = org_entity(part_iv_c_2c_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_2c_org)

In [35]:
part_iv_c_3_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-3.txt')
part_iv_c_3_dictionary_org = entity_extraction(part_iv_c_3_formatted_org)
part_iv_c_3_org = org_entity(part_iv_c_3_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_3_org)

In [36]:
part_iv_c_4_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-4.txt')
part_iv_c_4_dictionary_org = entity_extraction(part_iv_c_4_formatted_org)
part_iv_c_4_org = org_entity(part_iv_c_4_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_4_org)

In [37]:
part_iv_c_5_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-5.txt')
part_iv_c_5_dictionary_org = entity_extraction(part_iv_c_5_formatted_org)
part_iv_c_5_org = org_entity(part_iv_c_5_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_5_org)

In [38]:
part_iv_c_6a_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-6a.txt')
part_iv_c_6a_dictionary_org = entity_extraction(part_iv_c_6a_formatted_org)
part_iv_c_6a_org = org_entity(part_iv_c_6a_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_6a_org)

In [39]:
part_iv_c_6b_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-6b.txt')
part_iv_c_6b_dictionary_org = entity_extraction(part_iv_c_6b_formatted_org)
part_iv_c_6b_org = org_entity(part_iv_c_6b_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_6b_org)

In [40]:
part_iv_c_6c_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-6c.txt')
part_iv_c_6c_dictionary_org = entity_extraction(part_iv_c_6c_formatted_org)
part_iv_c_6c_org = org_entity(part_iv_c_6c_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_6c_org)

In [41]:
part_iv_c_7a_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-7a.txt')
part_iv_c_7a_dictionary_org = entity_extraction(part_iv_c_7a_formatted_org)
part_iv_c_7a_org = org_entity(part_iv_c_7a_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_7a_org)

In [42]:
part_iv_c_7b_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-7b.txt')
part_iv_c_7b_dictionary_org = entity_extraction(part_iv_c_7b_formatted_org)
part_iv_c_7b_org = org_entity(part_iv_c_7b_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_7b_org)

In [43]:
part_iv_c_8_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-8.txt')
part_iv_c_8_dictionary_org = entity_extraction(part_iv_c_8_formatted_org)
part_iv_c_8_org = org_entity(part_iv_c_8_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_8_org)

In [44]:
part_iv_c_9a_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-9a.txt')
part_iv_c_9a_dictionary_org = entity_extraction(part_iv_c_9a_formatted_org)
part_iv_c_9a_org = org_entity(part_iv_c_9a_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_9a_org)

In [45]:
part_iv_c_9b_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-9b.txt')
part_iv_c_9b_dictionary_org = entity_extraction(part_iv_c_9b_formatted_org)
part_iv_c_9b_org = org_entity(part_iv_c_9b_dictionary_org['ORG'])
main_df = concat_df(main_df, part_iv_c_9b_org)

In [46]:
part_v_a_vol_1a_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-A-Vol-IA.txt')
part_v_a_vol_1a_dictionary_org = entity_extraction(part_v_a_vol_1a_formatted_org)
part_v_a_vol_1a_org = org_entity(part_v_a_vol_1a_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_a_vol_1a_org)

In [47]:
part_v_a_vol_1b_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-A-Vol-IB.txt')
part_v_a_vol_1b_dictionary_org = entity_extraction(part_v_a_vol_1b_formatted_org)
part_v_a_vol_1b_org = org_entity(part_v_a_vol_1b_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_a_vol_1b_org)

In [48]:
part_v_a_vol_1c_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-A-Vol-IC.txt')
part_v_a_vol_1c_dictionary_org = entity_extraction(part_v_a_vol_1c_formatted_org)
part_v_a_vol_1c_org = org_entity(part_v_a_vol_1c_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_a_vol_1c_org)

In [49]:
part_v_a_vol_2d_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-A-Vol-IID.txt')
part_v_a_vol_2d_dictionary_org = entity_extraction(part_v_a_vol_2d_formatted_org)
part_v_a_vol_2d_org = org_entity(part_v_a_vol_2d_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_a_vol_2d_org)

In [50]:
part_v_b_1_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-1.txt')
part_v_b_1_dictionary_org = entity_extraction(part_v_b_1_formatted_org)
part_v_b_1_org = org_entity(part_v_b_1_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_b_1_org)

In [51]:
part_v_b_2a_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-2a.txt')
part_v_b_2a_dictionary_org = entity_extraction(part_v_b_2a_formatted_org)
part_v_b_2a_org = org_entity(part_v_b_2a_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_b_2a_org)

In [52]:
part_v_b_2b_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-2b.txt')
part_v_b_2b_dictionary_org = entity_extraction(part_v_b_2b_formatted_org)
part_v_b_2b_org = org_entity(part_v_b_2b_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_b_2b_org)

In [53]:
part_v_b_3a_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-3a.txt')
part_v_b_3a_dictionary_org = entity_extraction(part_v_b_3a_formatted_org)
part_v_b_3a_org = org_entity(part_v_b_3a_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_b_3a_org)

In [54]:
part_v_b_3b_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-3b.txt')
part_v_b_3b_dictionary_org = entity_extraction(part_v_b_3b_formatted_org)
part_v_b_3b_org = org_entity(part_v_b_3b_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_b_3b_org)

In [55]:
part_v_b_3c_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-3c.txt')
part_v_b_3c_dictionary_org = entity_extraction(part_v_b_3c_formatted_org)
part_v_b_3c_org = org_entity(part_v_b_3c_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_b_3c_org)

In [56]:
part_v_b_3d_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-3d.txt')
part_v_b_3d_dictionary_org = entity_extraction(part_v_b_3d_formatted_org)
part_v_b_3d_org = org_entity(part_v_b_3d_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_b_3d_org)

In [57]:
part_v_b_4_book_1_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-4-Book-I.txt')
part_v_b_4_book_1_dictionary_org = entity_extraction(part_v_b_4_book_1_formatted_org)
part_v_b_4_book_1_org = org_entity(part_v_b_4_book_1_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_b_4_book_1_org)

In [58]:
part_v_b_4_book_2_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-4-Book-II.txt')
part_v_b_4_book_2_dictionary_org = entity_extraction(part_v_b_4_book_2_formatted_org)
part_v_b_4_book_2_org = org_entity(part_v_b_4_book_2_dictionary_org['ORG'])
main_df = concat_df(main_df, part_v_b_4_book_2_org)

In [59]:
part_vi_a_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-A.txt')
part_vi_a_dictionary_org = entity_extraction(part_vi_a_formatted_org)
part_vi_a_org = org_entity(part_vi_a_dictionary_org['ORG'])
main_df = concat_df(main_df, part_vi_a_org)

In [60]:
part_vi_b_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-B.txt')
part_vi_b_dictionary_org = entity_extraction(part_vi_b_formatted_org)
part_vi_b_org = org_entity(part_vi_b_dictionary_org['ORG'])
main_df = concat_df(main_df, part_vi_b_org)

In [61]:
part_vi_c1_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-C-1.txt')
part_vi_c1_dictionary_org = entity_extraction(part_vi_c1_formatted_org)
part_vi_c1_org = org_entity(part_vi_c1_dictionary_org['ORG'])
main_df = concat_df(main_df, part_vi_c1_org)

In [62]:
part_vi_c2_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-C-2.txt')
part_vi_c2_dictionary_org = entity_extraction(part_vi_c2_formatted_org)
part_vi_c2_org = org_entity(part_vi_c2_dictionary_org['ORG'])
main_df = concat_df(main_df, part_vi_c2_org)

In [63]:
part_vi_c3_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-C-3.txt')
part_vi_c3_dictionary_org = entity_extraction(part_vi_c3_formatted_org)
part_vi_c3_org = org_entity(part_vi_c3_dictionary_org['ORG'])
main_df = concat_df(main_df, part_vi_c3_org)

In [64]:
part_vi_c4_formatted_org = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-C-4.txt')
part_vi_c4_dictionary_org = entity_extraction(part_vi_c4_formatted_org)
part_vi_c4_org = org_entity(part_vi_c4_dictionary_org['ORG'])
main_df = concat_df(main_df, part_vi_c4_org)

In [65]:
main_df

,Organizations,Count,Organizations,Count,Organizations,Count,Organizations,Count,Organizations,Count,...,Organizations,Count,Organizations,Count,Organizations,Count,Organizations,Count,Organizations,Count
0,A. Public Statements,1.0,A. Public Statements,1.0,8 1954,1.0,A-16 a.,1.0,83d Congress,2.0,...,1 Department,1.0,A. 132,1.0,1967 Estabrook,1.0,& troop,2.0,& U. S. (,1.0
1,A. U.S. France,1.0,A. U.S. France,1.0,A-14 DEPARTMENT OF,1.0,A-24 4,1.0,A-19 d. UN,1.0,...,1 Set,1.0,ACA,1.0,A. Presented,1.0,1967Harrison Salisbury,1.0,10GVN,1.0
2,A. U.S. Military Planning,1.0,A. U.S. Military Planning,1.0,A-17 d. French Intransigence,1.0,A-25 b. French,1.0,A-29(O,1.0,...,1965Tab'CC,1.0,AFP,4.0,AP,1.0,A. Position,1.0,1954 Geneva Accords,1.0
3,ASDISA,1.0,ASDISA,1.0,A-35 b. Military Assistance Program,1.0,A-25 c. United Action,1.0,A-3448,1.0,...,4A Nt,1.0,AMEMBASSY,1.0,AP Dispatch,1.0,AB,1.0,1967 State,1.0
4,Announced Position Statements C. Histories of ...,1.0,Announced Position Statements C. Histories of ...,1.0,A-35 c.,1.0,A-34 a. United,1.0,A-7,1.0,...,ABC Television's,1.0,ANALYSIS,1.0,Administration,3.0,ASAP,1.0,1967 Trinh,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
main_df.columns = 'Index', "Index_Count", 'Part-I', 'Part-I_Count', 'Part-2', 'Part-2_Count', 'Part-3', 'Part-3_Count', 'Part-IV-A-1', 'Part-IV-A-1_Count','Part-IV-A-2', 'Part-IV-A-2_Count','Part-IV-A-3', 'Part-IV-A-3_Count', 'Part-IV-A-4', 'Part-IV-A-4_Count','Part-IV-A-5', 'Part-IV-A-5_Count','Part-IV-B-1', 'Part-IV-B-1_Count','Part-IV-B-2', 'Part-IV-B-2_Count','Part-IV-B-3', 'Part-IV-B-3_Count','Part-IV-B-4', 'Part-IV-B-4_Count','Part-IV-B-5', 'Part-IV-B-5_Count','Part-IV-C-1', 'Part-IV-C-1_Count','Part-IV-C-2a', 'Part-IV-C-2a_Count','Part-IV-C-2b', 'Part-IV-C-2b_Count','Part-IV-C-2c', 'Part-IV-C-2c_Count','Part-IV-C-3', 'Part-IV-C-3_Count','Part-IV-C-4', 'Part-IV-C-4_Count','Part-IV-C-5', 'Part-IV-C-5_Count','Part-IV-C-6a', 'Part-IV-C-6a_Count','Part-IV-C-6b', 'Part-IV-C-6b_Count','Part-IV-C-6c', 'Part-IV-C-6c_Count','Part-IV-C-7a', 'Part-IV-C-7a_Count','Part-IV-C-7b', 'Part-IV-C-7b_Count','Part-IV-C-8', 'Part-IV-C-8_Count','Part-IV-C-9a', 'Part-IV-C-9a_Count','Part-IV-C-9b', 'Part-IV-C-9b_Count','Part-V-A-Vol-IA', 'Part-V-A-Vol-IA_Count','Part-V-A-Vol-IB', 'Part-V-A-Vol-IB_Count','Part-V-A-Vol-IC', 'Part-V-A-Vol-IC_Count','Part-V-A-Vol-IID', 'Part-V-A-Vol-IID_Count','Part-V-B-1', 'Part-V-B-1_Count','Part-V-B-2a', 'Part-V-B-2a_Count','Part-V-B-2b', 'Part-V-B-2b_Count','Part-V-B-3a', 'Part-V-B-3a_Count','Part-V-B-3b', 'Part-V-B-3b_Count','Part-V-B-3c', 'Part-V-B-3c_Count','Part-V-B-3d', 'Part-V-B-3d_Count','Part-V-B-4-Book-I', 'Part-V-B-4-Book-I_Count','Part-V-B-4-Book-II', 'Part-V-B-4-Book-II_Count','Part-VI-A', 'Part-VI-A_Count','Part-VI-B', 'Part-VI-B_Count','Part-VI-C-1', 'Part-VI-C-1_Count','Part-VI-C-2', 'Part-VI-C-2_Count','Part-VI-C-3', 'Part-VI-C-3_Count','Part-VI-C-4', 'Part-VI-C-4_Count'

In [67]:
organization_df = main_df

In [68]:
path = Path('organization_entity_data.csv')
organization_df.to_csv(path)

<H2> Formatting DF's for People </H2>

In [72]:
part_index_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-Index.txt')
part_index_dictionary_people = entity_extraction(part_index_formatted_people)
part_index_people = person_entity(part_index_dictionary_people['PERSON'])
# print(part_index_people) 

                      Person  Count
0                      Bundy      1
1       Clerk J The Pentagon      1
2                 I. Vietnam      1
3                    Kennedy      1
4           LESLIE H. CffiLB      1
5                   McNamara      1
6                  McNamaras      1
7                  Moby Dick      1
8         Morton H. Halperin      1
9             OSD Task Force      1
10  Pacification I965-I967 9      1
11                     Parts      1
12            Paul C. warnke      1
13        Robert S. McNamara      1
14             T. B. EDWARDS      1
15        Viet Minh Position      1
16       c. NovemberDecember      1
17         william C. Holzer      1


In [73]:
part_I_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-I.txt')
part_I_dictionary_people = entity_extraction(part_I_formatted_people)
part_I_people = person_entity(part_I_dictionary_people['PERSON'])

In [78]:
part_II_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-II.txt')
part_II_dictionary_people = entity_extraction(part_II_formatted_people)
part_II_people = person_entity(part_II_dictionary_people['PERSON'])

In [79]:
people_df = concat_df(part_index_people, part_I_people)
people_df = concat_df(people_df, part_II_people)
# people_df

In [80]:
part_III_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-III.txt')
part_III_dictionary_people = entity_extraction(part_III_formatted_people)
part_III_people = person_entity(part_III_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_III_people)

In [82]:
part_iv_a_1_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-1.txt')
part_iv_a_1_dictionary_people = entity_extraction(part_iv_a_1_formatted_people)
part_iv_a_1_people = person_entity(part_iv_a_1_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_a_1_people)

In [83]:
part_iv_a_2_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-2.txt')
part_iv_a_2_dictionary_people = entity_extraction(part_iv_a_2_formatted_people)
part_iv_a_2_people = person_entity(part_iv_a_2_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_a_2_people)

In [84]:
part_iv_a_3_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-3.txt')
part_iv_a_3_dictionary_people = entity_extraction(part_iv_a_3_formatted_people)
part_iv_a_3_people = person_entity(part_iv_a_3_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_a_3_people)

In [85]:
part_iv_a_4_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-4.txt')
part_iv_a_4_dictionary_people = entity_extraction(part_iv_a_4_formatted_people)
part_iv_a_4_people = person_entity(part_iv_a_4_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_a_4_people)

In [86]:
part_iv_a_5_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-A-5.txt')
part_iv_a_5_dictionary_people = entity_extraction(part_iv_a_5_formatted_people)
part_iv_a_5_people = person_entity(part_iv_a_5_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_a_5_people)

In [87]:
part_iv_b_1_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-1.txt')
part_iv_b_1_dictionary_people = entity_extraction(part_iv_b_1_formatted_people)
part_iv_b_1_people = person_entity(part_iv_b_1_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_b_1_people)

In [88]:
part_iv_b_2_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-2.txt')
part_iv_b_2_dictionary_people = entity_extraction(part_iv_b_2_formatted_people)
part_iv_b_2_people = person_entity(part_iv_b_2_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_b_2_people)

In [89]:
part_iv_b_3_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-3.txt')
part_iv_b_3_dictionary_people = entity_extraction(part_iv_b_3_formatted_people)
part_iv_b_3_people = person_entity(part_iv_b_3_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_b_3_people)

In [90]:
part_iv_b_4_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-4.txt')
part_iv_b_4_dictionary_people = entity_extraction(part_iv_b_4_formatted_people)
part_iv_b_4_people = person_entity(part_iv_b_4_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_b_4_people)

In [91]:
part_iv_b_5_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-B-5.txt')
part_iv_b_5_dictionary_people = entity_extraction(part_iv_b_5_formatted_people)
part_iv_b_5_people = person_entity(part_iv_b_5_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_b_5_people)

In [92]:
part_iv_c_1_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-1.txt')
part_iv_c_1_dictionary_people = entity_extraction(part_iv_c_1_formatted_people)
part_iv_c_1_people = person_entity(part_iv_c_1_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_1_people)

In [93]:
part_iv_c_2a_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-2a.txt')
part_iv_c_2a_dictionary_people = entity_extraction(part_iv_c_2a_formatted_people)
part_iv_c_2a_people = person_entity(part_iv_c_2a_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_2a_people)

In [94]:
part_iv_c_2b_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-2b.txt')
part_iv_c_2b_dictionary_people = entity_extraction(part_iv_c_2b_formatted_people)
part_iv_c_2b_people = person_entity(part_iv_c_2b_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_2b_people)

In [95]:
part_iv_c_2c_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-2c.txt')
part_iv_c_2c_dictionary_people = entity_extraction(part_iv_c_2c_formatted_people)
part_iv_c_2c_people = person_entity(part_iv_c_2c_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_2c_people)

In [96]:
part_iv_c_3_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-3.txt')
part_iv_c_3_dictionary_people = entity_extraction(part_iv_c_3_formatted_people)
part_iv_c_3_people = person_entity(part_iv_c_3_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_3_people)

In [97]:
part_iv_c_4_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-4.txt')
part_iv_c_4_dictionary_people = entity_extraction(part_iv_c_4_formatted_people)
part_iv_c_4_people = person_entity(part_iv_c_4_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_4_people)

In [98]:
part_iv_c_5_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-5.txt')
part_iv_c_5_dictionary_people = entity_extraction(part_iv_c_5_formatted_people)
part_iv_c_5_people = person_entity(part_iv_c_5_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_5_people)

In [99]:
part_iv_c_6a_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-6a.txt')
part_iv_c_6a_dictionary_people = entity_extraction(part_iv_c_6a_formatted_people)
part_iv_c_6a_people = person_entity(part_iv_c_6a_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_6a_people)

In [100]:
part_iv_c_6b_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-6b.txt')
part_iv_c_6b_dictionary_people = entity_extraction(part_iv_c_6b_formatted_people)
part_iv_c_6b_people = person_entity(part_iv_c_6b_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_6b_people)

In [101]:
part_iv_c_6c_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-6c.txt')
part_iv_c_6c_dictionary_people = entity_extraction(part_iv_c_6c_formatted_people)
part_iv_c_6c_people = person_entity(part_iv_c_6c_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_6c_people)

In [102]:
part_iv_c_7a_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-7a.txt')
part_iv_c_7a_dictionary_people = entity_extraction(part_iv_c_7a_formatted_people)
part_iv_c_7a_people = person_entity(part_iv_c_7a_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_7a_people)

In [103]:
part_iv_c_7b_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-7b.txt')
part_iv_c_7b_dictionary_people = entity_extraction(part_iv_c_7b_formatted_people)
part_iv_c_7b_people = person_entity(part_iv_c_7b_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_7b_people)

In [104]:
part_iv_c_8_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-8.txt')
part_iv_c_8_dictionary_people = entity_extraction(part_iv_c_8_formatted_people)
part_iv_c_8_people = person_entity(part_iv_c_8_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_8_people)

In [105]:
part_iv_c_9a_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-9a.txt')
part_iv_c_9a_dictionary_people = entity_extraction(part_iv_c_9a_formatted_people)
part_iv_c_9a_people = person_entity(part_iv_c_9a_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_9a_people)

In [106]:
part_iv_c_9b_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-IV-C-9b.txt')
part_iv_c_9b_dictionary_people = entity_extraction(part_iv_c_9b_formatted_people)
part_iv_c_9b_people = person_entity(part_iv_c_9b_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_iv_c_9b_people)

In [107]:
part_v_a_vol_ia_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-A-Vol-IA.txt')
part_v_a_vol_ia_dictionary_people = entity_extraction(part_v_a_vol_ia_formatted_people)
part_v_a_vol_ia_people = person_entity(part_v_a_vol_ia_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_a_vol_ia_people)

In [108]:
part_v_a_vol_ib_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-A-Vol-IB.txt')
part_v_a_vol_ib_dictionary_people = entity_extraction(part_v_a_vol_ib_formatted_people)
part_v_a_vol_ib_people = person_entity(part_v_a_vol_ib_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_a_vol_ib_people)

In [109]:
part_v_a_vol_ic_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-A-Vol-IC.txt')
part_v_a_vol_ic_dictionary_people = entity_extraction(part_v_a_vol_ic_formatted_people)
part_v_a_vol_ic_people = person_entity(part_v_a_vol_ic_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_a_vol_ic_people)

In [110]:
part_v_a_vol_2d_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-A-Vol-IID.txt')
part_v_a_vol_2d_dictionary_people = entity_extraction(part_v_a_vol_2d_formatted_people)
part_v_a_vol_2d_people = person_entity(part_v_a_vol_2d_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_a_vol_2d_people)

In [111]:
part_v_b_1_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-1.txt')
part_v_b_1_dictionary_people = entity_extraction(part_v_b_1_formatted_people)
part_v_b_1_people = person_entity(part_v_b_1_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_b_1_people)

In [112]:
part_v_b_2a_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-2a.txt')
part_v_b_2a_dictionary_people = entity_extraction(part_v_b_2a_formatted_people)
part_v_b_2a_people = person_entity(part_v_b_2a_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_b_2a_people)

In [113]:
part_v_b_2b_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-2b.txt')
part_v_b_2b_dictionary_people = entity_extraction(part_v_b_2b_formatted_people)
part_v_b_2b_people = person_entity(part_v_b_2b_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_b_2b_people)

In [114]:
part_v_b_3a_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-3a.txt')
part_v_b_3a_dictionary_people = entity_extraction(part_v_b_3a_formatted_people)
part_v_b_3a_people = person_entity(part_v_b_3a_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_b_3a_people)

In [115]:
part_v_b_3b_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-3b.txt')
part_v_b_3b_dictionary_people = entity_extraction(part_v_b_3b_formatted_people)
part_v_b_3b_people = person_entity(part_v_b_3b_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_b_3b_people)

In [116]:
part_v_b_3c_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-3c.txt')
part_v_b_3c_dictionary_people = entity_extraction(part_v_b_3c_formatted_people)
part_v_b_3c_people = person_entity(part_v_b_3c_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_b_3c_people)

In [117]:
part_v_b_3d_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-3d.txt')
part_v_b_3d_dictionary_people = entity_extraction(part_v_b_3d_formatted_people)
part_v_b_3d_people = person_entity(part_v_b_3d_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_b_3d_people)

In [118]:
part_v_b_4_book_1_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-4-Book-I.txt')
part_v_b_4_book_1_dictionary_people = entity_extraction(part_v_b_4_book_1_formatted_people)
part_v_b_4_book_1_people = person_entity(part_v_b_4_book_1_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_b_4_book_1_people)

In [119]:
part_v_b_4_book_2_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-V-B-4-Book-II.txt')
part_v_b_4_book_2_dictionary_people = entity_extraction(part_v_b_4_book_2_formatted_people)
part_v_b_4_book_2_people = person_entity(part_v_b_4_book_2_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_v_b_4_book_2_people)

In [120]:
part_vi_a_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-A.txt')
part_vi_a_dictionary_people = entity_extraction(part_vi_a_formatted_people)
part_vi_a_people = person_entity(part_vi_a_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_vi_a_people)

In [121]:
part_vi_b_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-B.txt')
part_vi_b_dictionary_people = entity_extraction(part_vi_b_formatted_people)
part_vi_b_people = person_entity(part_vi_b_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_vi_b_people)

In [122]:
part_vi_c_1_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-C-1.txt')
part_vi_c_1_dictionary_people = entity_extraction(part_vi_c_1_formatted_people)
part_vi_c_1_people = person_entity(part_vi_c_1_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_vi_c_1_people)

In [124]:
part_vi_c_2_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-C-2.txt')
part_vi_c_2_dictionary_people = entity_extraction(part_vi_c_2_formatted_people)
part_vi_c_2_people = person_entity(part_vi_c_2_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_vi_c_2_people)

In [125]:
part_vi_c_3_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-C-3.txt')
part_vi_c_3_dictionary_people = entity_extraction(part_vi_c_3_formatted_people)
part_vi_c_3_people = person_entity(part_vi_c_3_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_vi_c_3_people)

In [126]:
part_vi_c_4_formatted_people = prepare_document('../Cleaned_Pentagon_Papers_text_files/Cleaned_Pentagon-Papers-VI-C-4.txt')
part_vi_c_4_dictionary_people = entity_extraction(part_vi_c_4_formatted_people)
part_vi_c_4_people = person_entity(part_vi_c_4_dictionary_people['PERSON'])
people_df = concat_df(people_df, part_vi_c_4_people)

In [127]:
people_df

,Person,Count,Person,Count,Person,Count,Person,Count,Person,Count,...,Person,Count,Person,Count,Person,Count,Person,Count,Person,Count
0,Bundy,1.0,Bundy,1.0,-- Bao Dai,1.0,A-10 e.,1.0,53- JCS Memorandum,1.0,...,'Ash,1.0,'Nailer,1.0,9-,1.0,A.,1.0,-He,1.0
1,Clerk J The Pentagon,1.0,CSD Task Force,1.0,A-13,1.0,A-12 A-4III,1.0,A-10 h. Manila Conference,1.0,...,(,1.0,9-,1.0,9- Lewandowski,1.0,A. Begin,1.0,.which,1.0
2,I. Vietnam,1.0,Clerk J The Pentagon,1.0,A-13 c.,1.0,A-16 b.,1.0,A-13,1.0,...,--South Viet-Nam,1.0,9- X,1.0,A. DOrlandi,1.0,A. Kosygins,1.0,1967 Kissinger,2.0
3,Kennedy,1.0,I. Vietnam,1.0,A-38 b. Expectation,1.0,A-16 c.,1.0,A-15 a. Introductory Articles Alike,1.0,...,-0,1.0,ARNOLD SMITH,1.0,A. Lewandowski,1.0,Akalovsky,7.0,2.Nilsson,1.0
4,LESLIE H. CffiLB,1.0,JC TMT,1.0,A-39 c. American Policy,1.0,A-19 b.,1.0,A-15Asian,1.0,...,-which al 1,1.0,Adalbert de Segonzac,1.0,A. Poles,2.0,Aleksei N. Kosygin,1.0,5THE,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
people_df.columns = 'Index', "Index_Count", 'Part-I', 'Part-I_Count', 'Part-2', 'Part-2_Count', 'Part-3', 'Part-3_Count', 'Part-IV-A-1', 'Part-IV-A-1_Count','Part-IV-A-2', 'Part-IV-A-2_Count','Part-IV-A-3', 'Part-IV-A-3_Count', 'Part-IV-A-4', 'Part-IV-A-4_Count','Part-IV-A-5', 'Part-IV-A-5_Count','Part-IV-B-1', 'Part-IV-B-1_Count','Part-IV-B-2', 'Part-IV-B-2_Count','Part-IV-B-3', 'Part-IV-B-3_Count','Part-IV-B-4', 'Part-IV-B-4_Count','Part-IV-B-5', 'Part-IV-B-5_Count','Part-IV-C-1', 'Part-IV-C-1_Count','Part-IV-C-2a', 'Part-IV-C-2a_Count','Part-IV-C-2b', 'Part-IV-C-2b_Count','Part-IV-C-2c', 'Part-IV-C-2c_Count','Part-IV-C-3', 'Part-IV-C-3_Count','Part-IV-C-4', 'Part-IV-C-4_Count','Part-IV-C-5', 'Part-IV-C-5_Count','Part-IV-C-6a', 'Part-IV-C-6a_Count','Part-IV-C-6b', 'Part-IV-C-6b_Count','Part-IV-C-6c', 'Part-IV-C-6c_Count','Part-IV-C-7a', 'Part-IV-C-7a_Count','Part-IV-C-7b', 'Part-IV-C-7b_Count','Part-IV-C-8', 'Part-IV-C-8_Count','Part-IV-C-9a', 'Part-IV-C-9a_Count','Part-IV-C-9b', 'Part-IV-C-9b_Count','Part-V-A-Vol-IA', 'Part-V-A-Vol-IA_Count','Part-V-A-Vol-IB', 'Part-V-A-Vol-IB_Count','Part-V-A-Vol-IC', 'Part-V-A-Vol-IC_Count','Part-V-A-Vol-IID', 'Part-V-A-Vol-IID_Count','Part-V-B-1', 'Part-V-B-1_Count','Part-V-B-2a', 'Part-V-B-2a_Count','Part-V-B-2b', 'Part-V-B-2b_Count','Part-V-B-3a', 'Part-V-B-3a_Count','Part-V-B-3b', 'Part-V-B-3b_Count','Part-V-B-3c', 'Part-V-B-3c_Count','Part-V-B-3d', 'Part-V-B-3d_Count','Part-V-B-4-Book-I', 'Part-V-B-4-Book-I_Count','Part-V-B-4-Book-II', 'Part-V-B-4-Book-II_Count','Part-VI-A', 'Part-VI-A_Count','Part-VI-B', 'Part-VI-B_Count','Part-VI-C-1', 'Part-VI-C-1_Count','Part-VI-C-2', 'Part-VI-C-2_Count','Part-VI-C-3', 'Part-VI-C-3_Count','Part-VI-C-4', 'Part-VI-C-4_Count'

In [129]:
people_df

,Index,Index_Count,Part-I,Part-I_Count,Part-2,Part-2_Count,Part-3,Part-3_Count,Part-IV-A-1,Part-IV-A-1_Count,...,Part-VI-B,Part-VI-B_Count,Part-VI-C-1,Part-VI-C-1_Count,Part-VI-C-2,Part-VI-C-2_Count,Part-VI-C-3,Part-VI-C-3_Count,Part-VI-C-4,Part-VI-C-4_Count
0,Bundy,1.0,Bundy,1.0,-- Bao Dai,1.0,A-10 e.,1.0,53- JCS Memorandum,1.0,...,'Ash,1.0,'Nailer,1.0,9-,1.0,A.,1.0,-He,1.0
1,Clerk J The Pentagon,1.0,CSD Task Force,1.0,A-13,1.0,A-12 A-4III,1.0,A-10 h. Manila Conference,1.0,...,(,1.0,9-,1.0,9- Lewandowski,1.0,A. Begin,1.0,.which,1.0
2,I. Vietnam,1.0,Clerk J The Pentagon,1.0,A-13 c.,1.0,A-16 b.,1.0,A-13,1.0,...,--South Viet-Nam,1.0,9- X,1.0,A. DOrlandi,1.0,A. Kosygins,1.0,1967 Kissinger,2.0
3,Kennedy,1.0,I. Vietnam,1.0,A-38 b. Expectation,1.0,A-16 c.,1.0,A-15 a. Introductory Articles Alike,1.0,...,-0,1.0,ARNOLD SMITH,1.0,A. Lewandowski,1.0,Akalovsky,7.0,2.Nilsson,1.0
4,LESLIE H. CffiLB,1.0,JC TMT,1.0,A-39 c. American Policy,1.0,A-19 b.,1.0,A-15Asian,1.0,...,-which al 1,1.0,Adalbert de Segonzac,1.0,A. Poles,2.0,Aleksei N. Kosygin,1.0,5THE,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
path = Path('people_entity_data.csv')
people_df.to_csv(path)